In [31]:
#loading libraries 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.ensemble import VotingClassifier
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn. preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [32]:
df_train=pd.read_csv("Train_Clean.csv")
df_test=pd.read_csv("Test_Clean.csv")


In [33]:
x_train=df_train["Tweets"]
y_train=df_train["label"]
x_test=df_test["Tweet"]
y_test=df_test["Threat"]
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((3564,), (935,), (3564,), (935,))

In [34]:
#vectorizing the urdu statements using the Term frequency and inverse document frequency
from sklearn.feature_extraction.text import TfidfVectorizer
max_feature_num = 50000
vectorizer = TfidfVectorizer(max_features=max_feature_num)
train_vecs = vectorizer.fit_transform(x_train)
test_vecs = TfidfVectorizer(max_features=max_feature_num, vocabulary=vectorizer.vocabulary_).fit_transform(x_test)
# visualizing the shape of train and test vecs
train_vecs.shape,test_vecs.shape


((3564, 7642), (935, 7642))

In [35]:
estimators=[]
#adding weak 6 gnb
estimators.append(("GNB1",GaussianNB()))
estimators.append(("GNB2",GaussianNB()))
estimators.append(("GNB3",GaussianNB()))
estimators.append(("GNB4",GaussianNB()))
estimators.append(("GNB5",GaussianNB()))
estimators.append(("GNB6",GaussianNB()))
#adding 6 weak svc 
estimators.append(("LSVC1",LinearSVC()))
estimators.append(("LSVC2",LinearSVC()))
estimators.append(("LSVC3",LinearSVC()))
estimators.append(("LSVC4",LinearSVC()))
estimators.append(("LSVC5",LinearSVC()))
estimators.append(("LSVC6",LinearSVC()))
#adding 6 weak dtc
estimators.append(("DTC1",DecisionTreeClassifier()))
estimators.append(("DTC2",DecisionTreeClassifier()))
estimators.append(("DTC3",DecisionTreeClassifier()))
estimators.append(("DTC4",DecisionTreeClassifier()))
estimators.append(("DTC5",DecisionTreeClassifier()))
estimators.append(("DTC6",DecisionTreeClassifier()))
#adding 6 weak knn
estimators.append(("KNN1",KNeighborsClassifier()))
estimators.append(("KNN2",KNeighborsClassifier()))
estimators.append(("KNN3",KNeighborsClassifier()))
estimators.append(("KNN4",KNeighborsClassifier()))
estimators.append(("KNN5",KNeighborsClassifier()))
estimators.append(("KNN6",KNeighborsClassifier()))
#adding 6 weak logistic regression classifier
estimators.append(("LR1",LogisticRegression()))
estimators.append(("LR2",LogisticRegression()))
estimators.append(("LR3",LogisticRegression()))
estimators.append(("LR4",LogisticRegression()))
estimators.append(("LR5",LogisticRegression()))
estimators.append(("LR6",LogisticRegression()))
#adding weak Multinomial Naive Bayes Classifier
estimators.append(("MNB1",MultinomialNB()))
estimators.append(("MNB2",MultinomialNB()))
estimators.append(("MNB3",MultinomialNB()))
estimators.append(("MNB4",MultinomialNB()))
estimators.append(("MNB5",MultinomialNB()))
estimators.append(("MNB6",MultinomialNB()))
#adding 6 weak LDA classfiers 
estimators.append(("LDA1",LinearDiscriminantAnalysis()))
estimators.append(("LDA2",LinearDiscriminantAnalysis()))
estimators.append(("LDA3",LinearDiscriminantAnalysis()))
estimators.append(("LDA4",LinearDiscriminantAnalysis()))
estimators.append(("LDA5",LinearDiscriminantAnalysis()))
estimators.append(("LDA6",LinearDiscriminantAnalysis()))

In [36]:
#defining the ensemble model
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
np.random.seed(42)
def build_model(model,xtrain,ytrain,xtest,ytest):
    clf = model
    clf.fit(xtrain,ytrain)
    clf_predictions = clf.predict(xtest)
    acc = accuracy_score(ytest,clf_predictions)
    f_1_score = f1_score(ytest,clf_predictions,average="weighted")
    roc_auc=roc_auc_score(ytest,clf_predictions)
    result = {"accuracy:":acc,"f1_score":f_1_score,"ROC_AUC":roc_auc}
    return model,result
ensemble_model,result=build_model(VotingClassifier(estimators),train_vecs.toarray(),y_train,test_vecs.toarray(),y_test)

In [37]:
ensemble_model

VotingClassifier(estimators=[('GNB1', GaussianNB()), ('GNB2', GaussianNB()),
                             ('GNB3', GaussianNB()), ('GNB4', GaussianNB()),
                             ('GNB5', GaussianNB()), ('GNB6', GaussianNB()),
                             ('LSVC1', LinearSVC()), ('LSVC2', LinearSVC()),
                             ('LSVC3', LinearSVC()), ('LSVC4', LinearSVC()),
                             ('LSVC5', LinearSVC()), ('LSVC6', LinearSVC()),
                             ('DTC1', DecisionTreeClassifier()),
                             ('DTC2', DecisionTreeCla...
                             ('KNN1', KNeighborsClassifier()),
                             ('KNN2', KNeighborsClassifier()),
                             ('KNN3', KNeighborsClassifier()),
                             ('KNN4', KNeighborsClassifier()),
                             ('KNN5', KNeighborsClassifier()),
                             ('KNN6', KNeighborsClassifier()),
                             ('LR1', LogisticRegression()),
                             ('LR2', LogisticRegression()),
                             ('LR3', LogisticRegression()),
                             ('LR4', LogisticRegression()),
                             ('LR5', LogisticRegression()),
                             ('LR6', LogisticRegression()), ...])

In [38]:
result

{'accuracy:': 0.6887700534759358,
 'f1_score': 0.6950334877024269,
 'ROC_AUC': 0.6734086806780223}

In [43]:
#cross validation of ensemble technique
from sklearn.model_selection import KFold, cross_val_score
kf =KFold(n_splits=10, shuffle=True, random_state=42)
result_cv=cross_val_score(ensemble_model,train_vecs.toarray(),y_train,scoring="accuracy",cv=kf)

In [45]:
result_cv.mean()

0.7121290089069335

In [46]:
result_cv_f1=cross_val_score(ensemble_model,train_vecs.toarray(),y_train,scoring="f1",cv=kf)

In [47]:
result_cv_roc=cross_val_score(ensemble_model,train_vecs.toarray(),y_train,scoring="roc_auc",cv=kf)

/home/shirish/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/shirish/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 358, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/home/shirish/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 71, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'VotingClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/shirish/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/shirish/anaconda3/lib/python3.9/site-packages/skle

/home/shirish/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/home/shirish/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 358, in _score
    y_pred = method_caller(clf, "decision_function", X)
  File "/home/shirish/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 71, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
AttributeError: 'VotingClassifier' object has no attribute 'decision_function'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/shirish/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/shirish/anaconda3/lib/python3.9/site-packages/skle

KeyboardInterrupt: 

In [48]:
result_cv_f1.mean()

0.7145640745578099